# Clasificación de Dígitos MNIST - Versión Compatible

Versión actualizada para generar modelo compatible con versiones modernas de TensorFlow

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import math
import logging

logger = tf.get_logger()
logger.setLevel(logging.ERROR)

print('TensorFlow versión:', tf.__version__)

TensorFlow versión: 2.19.0


In [ ]:
print('Cargando el dataset MNIST...')

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

print('Dataset cargado exitosamente')

Cargando el dataset MNIST...
Dataset cargado exitosamente


In [ ]:
def normalize(images, labels):
    images = tf.cast(images, tf.float32)
    images /= 255
    return images, labels

train_dataset = ds_train.map(normalize, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = ds_test.map(normalize, num_parallel_calls=tf.data.AUTOTUNE)

print('Datos normalizados')

Datos normalizados


In [ ]:
print('Definiendo la estructura de la CNN...')

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28, 1)),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='softmax')
])

print('Modelo CNN definido')
model.summary()


Definiendo la estructura de la CNN...
Modelo CNN definido


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print('Modelo compilado')

Modelo compilado


In [ ]:
BATCHSIZE = 32

train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(ds_info.splits['train'].num_examples)
train_dataset = train_dataset.batch(BATCHSIZE)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

test_dataset = test_dataset.batch(BATCHSIZE)
test_dataset = test_dataset.cache()
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

print('Datos preparados para entrenamiento')

Datos preparados para entrenamiento


In [ ]:
print('Comenzando el entrenamiento del modelo (5 épocas)...')

history = model.fit(
    train_dataset,
    epochs=5,
    validation_data=test_dataset,
    verbose=1
)

print('Entrenamiento completado')

Comenzando el entrenamiento del modelo (5 épocas)...
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 63s 29ms/step - accuracy: 0.8861 - loss: 0.3587 - val_accuracy: 0.9855 - val_loss: 0.0441
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 54s 29ms/step - accuracy: 0.9811 - loss: 0.0619 - val_accuracy: 0.9891 - val_loss: 0.0321
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 54s 29ms/step - accuracy: 0.9870 - loss: 0.0408 - val_accuracy: 0.9908 - val_loss: 0.0304
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 28ms/step - accuracy: 0.9899 - loss: 0.0321 - val_accuracy: 0.9889 - val_loss: 0.0328
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 53s 28ms/step - accuracy: 0.9919 - loss: 0.0250 - val_accuracy: 0.9909 - val_loss: 0.0272
Entrenamiento completado


In [ ]:
print('Evaluando el modelo...')

test_loss, test_accuracy = model.evaluate(
    test_dataset,
    verbose=1
)

print(f'Precisión en datos de prueba: {test_accuracy:.4f}')

Evaluando el modelo...
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9915 - loss: 0.0247
Precisión en datos de prueba: 0.9909


In [ ]:
print('Guardando modelo compatible...')

model.save('modelo_MINST_completo.keras')

print('Modelo guardado como modelo_MINST_completo.keras')
print('Modelo compatible generado exitosamente!')

Guardando modelo compatible...
Modelo guardado como modelo_MINST_completo.keras
Modelo compatible generado exitosamente!


In [ ]:
print('Probando el modelo guardado...')

loaded_model = tf.keras.models.load_model('modelo_MINST_completo.keras')

loaded_test_loss, loaded_test_accuracy = loaded_model.evaluate(
    test_dataset,
    verbose=1
)

print(f'Precisión del modelo cargado: {loaded_test_accuracy:.4f}')
print('Modelo guardado y cargado correctamente!')

Probando el modelo guardado...
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9915 - loss: 0.0247
Precisión del modelo cargado: 0.9909
Modelo guardado y cargado correctamente!
